In [1]:
import tensorflow as tf
from tensorflow.python.keras.layers import Dense, Flatten, Conv2D
from tensorflow.python.keras import Model

physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    pass

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255. , x_test / 255.

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [3]:
# 채널 차원 추가
x_train = x_train[..., tf.newaxis].astype('float32') # ... : Ellipsis 객체 (생략 부호)
x_test = x_test[..., tf.newaxis].astype('float32')

print(x_train.shape)
print(x_test.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


- `tf.data`를 사용하여 'batch' 처리 및 shuffle

In [4]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len(x_train)).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).shuffle(len(x_test)).batch(32)

- 'model sub-classing'을 사용하여 keras 모델 빌드

In [5]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10)
        
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        x = self.d2(x)
        return x

In [6]:
model = MyModel()

- 'loss function' 과 'optimizer' 정의

In [7]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

- 모델의 'accuracy' 와 'loss'를 측정하기 위한 'metrics' 정의.
- 이 'metrics'는 epoch에 걸쳐 값을 누적하고 나서 전체 결과를 출력한다.

In [8]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

- `tf.GradientTape`을 사용하여 모델 학습

In [9]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
        
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)

In [10]:
@tf.function
def test_step(images, labels):
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)
    
    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [11]:
EPOCHS = 10

for epoch in range(EPOCHS):
    train_loss.reset_state() # 새 반복이 될 때마다 metrics 리셋. 굉장히 중요!
    train_accuracy.reset_state()
    test_loss.reset_state()
    test_accuracy.reset_state()
    
    for images, labels in train_ds:
        train_step(images, labels)
        
    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)
        
    print(
        f'Epoch {epoch + 1}, ',
        f'Loss: {train_loss.result()}, ',
        f'Accuracy: {train_accuracy.result() * 100}, ',
        f'Test Loss: {test_loss.result()}, ',
        f'Test Accuracy: {test_accuracy.result() * 100}'
    )

Epoch 1,  Loss: 0.13389046490192413,  Accuracy: 95.97833251953125,  Test Loss: 0.06536329537630081,  Test Accuracy: 97.86000061035156
Epoch 2,  Loss: 0.041421495378017426,  Accuracy: 98.72833251953125,  Test Loss: 0.052074942737817764,  Test Accuracy: 98.2699966430664
Epoch 3,  Loss: 0.022947918623685837,  Accuracy: 99.2699966430664,  Test Loss: 0.054011646658182144,  Test Accuracy: 98.38999938964844
Epoch 4,  Loss: 0.014135980047285557,  Accuracy: 99.54666137695312,  Test Loss: 0.05373052880167961,  Test Accuracy: 98.43999481201172
Epoch 5,  Loss: 0.008769259788095951,  Accuracy: 99.74666595458984,  Test Loss: 0.06321606785058975,  Test Accuracy: 98.27999877929688
Epoch 6,  Loss: 0.007814153097569942,  Accuracy: 99.73833465576172,  Test Loss: 0.07622847706079483,  Test Accuracy: 98.29000091552734
Epoch 7,  Loss: 0.006007640156894922,  Accuracy: 99.788330078125,  Test Loss: 0.07368568331003189,  Test Accuracy: 98.31999969482422
Epoch 8,  Loss: 0.004545972216874361,  Accuracy: 99.856666